### Set current directory

In [1]:
%cd /kaggle/working

### Import libraries

In [2]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

from tensorflow.keras.applications.vgg16 import preprocess_input

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### Set seed for reproducibility

In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Set dataset folders

In [4]:
# Dataset folders 
dataset_dir = '../input/anndl-dataset'
training_dir = os.path.join(dataset_dir, 'training')

### Class labels: 

1:Apple 

2:Blueberry 

3:Cherry 

4:Corn 

5:Grape 

6:Orange 

7:Peach 

8:Pepper 

9:Potato 

10:Raspberry 

11:Soybean 

12:Squash 

13:Strawberry 

14:Tomato

### Plot example images from dataset

In [5]:
# Plot example images from dataset
labels = ['Apple',              # 0
          'Blueberry',          # 1
          'Cherry',             # 2
          'Corn',               # 3
          'Grape',              # 4
          'Orange',             # 5
          'Peach',              # 6
          'Pepper',             # 7
          'Potato',             # 8
          'Raspberry',          # 9
          'Soybean',            # 10
          'Squash',             # 11
          'Strawberry',         # 12
          'Tomato']             # 13

num_row = len(labels)//3
num_col = len(labels)//num_row
fig, axes = plt.subplots(num_row, num_col, figsize=(3*num_row,5*num_col))
for i in range(num_row*num_col):
    if i < len(labels):
        class_imgs = next(os.walk('{}/training/{}/'.format(dataset_dir, labels[i])))[2]
        class_img = class_imgs[0]
        img = Image.open('{}/training/{}/{}'.format(dataset_dir, labels[i], class_img))
        ax = axes[i//num_col, i%num_col]
        ax.imshow(np.array(img))
        ax.set_title('{}'.format(labels[i]))
plt.tight_layout()
plt.show()

# Data Loader + Data Augmentation

##### ImageDataGenerator allows to perform data augmentation

```
tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
    height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=False, vertical_flip=False, rescale=None,
    preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
)
```

In [6]:
batch_size = 64

### Mean-centering and std-normalization of the training set is not performed becouse of the pre-processing on vgg16

In [7]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator with Data Augmentation
# no rescale value <- vgg16 automatic rescale the input in the first layer
training_data_gen = ImageDataGenerator(#featurewise_center=True, 
                                    #featurewise_std_normalization=True, 
                                    rotation_range=30, 
                                    height_shift_range=50, 
                                    width_shift_range=50, 
                                    zoom_range=0.3, 
                                    horizontal_flip=True, 
                                    vertical_flip=True, 
                                    fill_mode='reflect', 
                                    preprocessing_function=preprocess_input, 
                                    validation_split=0.15) 

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
training_set_gen = training_data_gen.flow_from_directory(directory=training_dir, 
                                                   target_size=(256,256), 
                                                   color_mode='rgb', 
                                                   classes=None, # can be set to labels 
                                                   class_mode='categorical', 
                                                   batch_size=batch_size, 
                                                   shuffle=True, 
                                                   subset='training', 
                                                   seed=seed)

validation_set_gen = training_data_gen.flow_from_directory(directory=training_dir, 
                                                   target_size=(256,256), 
                                                   color_mode='rgb', 
                                                   classes=None, # can be set to labels 
                                                   class_mode='categorical', 
                                                   batch_size=batch_size, 
                                                   shuffle=False, 
                                                   subset='validation',
                                                   seed=seed)

### Assigned label and target classes

In [8]:
print("Assigned labels")
print(training_set_gen.class_indices)
print()
print("Target classes")
print(training_set_gen.classes)

### Get a sample from dataset and show info

In [9]:
def get_next_batch(generator):
    batch = next(generator)

    image = batch[0]
    target = batch[1]

    print("(Input) image shape:", image.shape)
    print("Target shape:",target.shape)

    # Visualize only the first sample
    image = image[0]
    target = target[0]
    target_idx = np.argmax(target)
    print()
    print("Categorical label:", target)
    print("Label:", target_idx)
    print("Class name:", labels[target_idx])
    fig = plt.figure(figsize=(6, 4))
    plt.imshow(np.uint8(image))

    return batch

In [10]:
# Get a sample from dataset and show info
_ = get_next_batch(training_set_gen)

### Models metadata

In [11]:
input_shape = (256, 256, 3)
epochs = 200

# Transfer learning

In [12]:
# Download and plot the VGG16 model
supernet = tfk.applications.VGG16(
    include_top=False, # with false we remove the classifier from vgg, taking only the features extraction part
    weights="imagenet", # means that we want the weights of the network trained on the imagenet
    input_shape=input_shape # is the shape of the input with witch we will train the supernet
)
supernet.summary()
tfk.utils.plot_model(supernet)

# Supernetwork as feature extractor (1° phase, all the layers frozen)
#### (in the following 2° phase: Fine Tuning)
##### In this first phase we use the supernet as feature extractor, freezing all the imported layer. In the following and last phase we perform Fine Tuning, setting to learnable the last layer of the imported CNN, and keep freezeng the first N layer [N must be decided])

In [13]:
# Use the supernet as feature extractor
supernet.trainable = False

# Input layer
inputs = tfk.Input(shape=input_shape)

# Supernet layer
x = supernet(inputs)

# MaxPooling layer
x = tfkl.MaxPooling2D(name='MaxPooling2D')(x)

# Flattening layer
x = tfkl.Flatten(name='Flatten')(x)

# 1° hidden layer
x = tfkl.Dense(256, 
               activation='relu', 
               kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)

# 2° hidden layer
x = tfkl.Dense(256, 
               activation='relu', 
               kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)

# Output layer
outputs = tfkl.Dense(len(labels), 
                     activation='softmax', 
                     kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


# Connect input and output through the Model class
tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model, adam learning rate: 0.001 (in the second phase will be shrinked)
tl_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(0.001), metrics='accuracy')
tl_model.summary()

## Utility function to create callbacks (for checkpoint and earlystopping) and folders

In [14]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

    exps_dir = os.path.join('transfer_learning_experiments')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)

    callbacks = []

    # Model last checkpoint
    # ----------------
    ckpt_dir = os.path.join(exp_dir, 'lastckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'lastcp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch 
    callbacks.append(ckpt_callback)

    # Model best checkpoint
    # ----------------
    ckpt_dir = os.path.join(exp_dir, 'bestckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'bestcp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=True) # True to save only the best epoch 
    callbacks.append(ckpt_callback)

    # Visualize Learning on Tensorboard
    # ---------------------------------
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)

    # By default shows losses and metrics for both training and validation
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
    callbacks.append(tb_callback)

    # Early Stopping
    # --------------
    es_callback = tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=25, restore_best_weights=True)
    callbacks.append(es_callback)

    return callbacks

### Set class weight

In [15]:
# Set class weight

class_weights = [1.2817,2.7115,2.172,1.05,0.8685,0.7244,1.2961,1.6553,1.7686,4.7965,0.7836,2.2061,1.8816,0.2224]
class_weights = dict(enumerate(class_weights))

### Train the model

In [ ]:
# Create folders and callbacks
tl_callbacks = create_folders_and_callbacks(model_name='TransferLearning_first_step')

# Train the model
tl_history = tl_model.fit(x = training_set_gen, 
                          class_weight = class_weights, 
                          batch_size = batch_size, 
                          epochs = epochs, 
                          validation_data = validation_set_gen, 
                          callbacks = tl_callbacks 
                         ).history

### Plot the training

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(standard_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(standard_history['val_loss'], label='Standard', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(standard_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(standard_history['val_accuracy'], label='Standard', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

### Confusion matrix on the validaion set

In [ ]:
tl_model = tfk.models.load_model('TransferLearningModel')
# Predict the test set with the CNN
predictions = tl_model.predict(validation_set_gen)
predictions.shape

# Compute the confusion matrix
cm = confusion_matrix(np.argmax(validation_set_gen, axis=-1), np.argmax(predictions, axis=-1))

# Compute the classification metrics
accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=list(labels.values()), yticklabels=list(labels.values()))
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

### Save the model

In [ ]:
# Save the best model
tl_model.save('TransferLearningModel')

# Fine Tuning (2° phase)

In [ ]:
# Re-load the model after transfer learning
ft_model = tfk.models.load_model('TransferLearningModel')
ft_model.summary()

### Now we keep frozen only the first N layer of the supernetwork, and set to trainable the other ones

In [ ]:
# Freeze first N layers, e.g., until 14th
for i, layer in enumerate(ft_model.get_layer('vgg16').layers[:14]):
    layer.trainable=False
for i, layer in enumerate(ft_model.get_layer('vgg16').layers):
    print(i, layer.name, layer.trainable)
ft_model.summary()

### We now re-compile the model shrinking the learning rate, to not "destroy" supernetwork weights

In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

### Fine-Tune the model

In [ ]:
# Create folders and callbacks
ft_callbacks = create_folders_and_callbacks(model_name='Fine_Tuning_2_step')

# Fine-tune the model
ft_history = ft_model.fit(x = training_set_gen, 
                          class_weight = class_weights, 
                          batch_size = batch_size, 
                          epochs = ephocs, 
                          validation_data = validation_set_gen, 
                          callbacks = ft_callbacks 
                         ).history

### Plot the training

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(standard_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(standard_history['val_loss'], label='Standard', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['loss'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(standard_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(standard_history['val_accuracy'], label='Standard', alpha=.8, color='#ff7f0e')
plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['accuracy'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

### Confusion Matrix on the validation set

In [ ]:
# Predict the test set with the CNN
predictions = ft_model.predict(validation_set_gen)
predictions.shape

# Compute the confusion matrix
cm = confusion_matrix(np.argmax(validation_set_gen, axis=-1), np.argmax(predictions, axis=-1))

# Compute the classification metrics
accuracy = accuracy_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1))
precision = precision_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
recall = recall_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
f1 = f1_score(np.argmax(y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=list(labels.values()), yticklabels=list(labels.values()))
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

#### Save the model

In [ ]:
ft_model.save('FineTuningModel')